#### Calculate and write out vmf, p-class, and PE from Rosi's MPAS simulations

Using modified code from Rosi's notebooks

Optimized by gemini.

### Main settings and cluster

In [1]:
import xarray as xr
import numpy as np
import dask
import dask.array as da

#### Start cluster and scale it up

In [2]:
# from ncar_jobqueue import NCARCluster
# cluster = PBSCluster()
# cluster.scale(4 * 9) # Ask for 4 x 9 workers
# cluster

from dask_jobqueue import PBSCluster
# Create a PBS cluster object
# cluster = PBSCluster(
#     job_name = 'dask-wk23-hpc',
#     cores = 1,
#     processes = 1,
#     log_directory = '/glade/derecho/scratch/rberrios/dask/',
#     local_directory = '/glade/derecho/scratch/rberrios/dask/',
#     resource_spec = 'select=1:ncpus=1:mem=10GB',
#     queue = 'casper',
#     walltime = '12:00:00',
#     interface = 'mgt'
# )
cluster = PBSCluster(
    job_name = 'dask-wk23-hpc',
    cores = 1,
    processes = 1,
    memory = '10GiB',
    account = 'UOKL0049',
    log_directory = '/glade/derecho/scratch/ruppert/dask/',
    local_directory = '/glade/derecho/scratch/ruppert/dask/',
    resource_spec = 'select=1:ncpus=1:mem=10GB',
    queue = 'casper',
    # queue = 'main',
    walltime = '12:00:00',
    interface = 'ext'
)
print(cluster.job_script())

#!/usr/bin/env bash

#PBS -N dask-wk23-hpc
#PBS -q casper
#PBS -A UOKL0049
#PBS -l select=1:ncpus=1:mem=10GB
#PBS -l walltime=12:00:00
#PBS -e /glade/derecho/scratch/ruppert/dask//
#PBS -o /glade/derecho/scratch/ruppert/dask//

/glade/work/ruppert/conda-envs/plotting/bin/python -m distributed.cli.dask_worker tcp://128.117.211.222:46297 --name dummy-name --nthreads 1 --memory-limit 10.00GiB --nanny --death-timeout 60 --local-directory /glade/derecho/scratch/ruppert/dask/ --interface ext



In [3]:
from dask.distributed import Client
client = Client(cluster) # Connect this local process to remote workers
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: http://128.117.211.222:8787/status,
Dashboard: http://128.117.211.222:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.211.222:46297,Workers: 0
Dashboard: http://128.117.211.222:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
# Scale the cluster to n workers (which will use n jobs here)
ncpu = 36
# ncpu = 24
# ncpu = 10
# ncpu = 18
cluster.scale(ncpu)

# Block progress until workers have spawned (typically only in demos and benchmarks!)
client.wait_for_workers(ncpu)

#### Set paths, read initial conditions, find tropical indexes

In [5]:
dx = "TC_3km"
grid_path = "/glade/work/rberrios/MPAS/aqua_sstmax10N_ASD/plus4K/TC_3km/x5.tropical_3km_10N.init.nc"

In [6]:
grid = xr.open_dataset(grid_path)
latCell = np.degrees(grid.latCell)
lonCell = np.degrees(grid.lonCell)
areaCell = grid.areaCell

#find indexes within desired latitudinal range
latbounds = [15, 20.0]
# latbounds = [0, 15.0]
ind_within_lat = np.where( (latCell >= latbounds[0]) & (latCell <= latbounds[1]) )[0]

areaCell_tropical = areaCell.isel(nCells=ind_within_lat)
# len(ind_within_lat)

### Functions

#### P-class function

In [7]:
# 
# Column-based precipitation classification algorithm designed for application on
# numerical model output.
# 
# It has been designed using WRF model output using the Thompson and Eidhammer
# (2014, JAS) microphysics scheme, which has 2 liquid and 3 frozen categories as
# listed and expected below.
# 
# Input:
# 
#       Q_INT: n-D array of vertically integrated hydrometeors as f(q, X), where
#               q(5) is the hydrometeor dimension, arranged as
#               ['QCLOUD', 'QRAIN', 'QICE', 'QSNOW', 'QGRAUP'] and X includes the
#               remaining (time and) spatial dimensions.
# Returns:
# 
#       C_TYPE: (n-2)-D array as f(X) with classification results:
#               0: non-cloud
#           Convective:
#               1: deep convective
#               2: congestus
#               3: shallow
#           Layered:
#               4: stratiform
#               5: anvil (weaker rainfall)
# 
# Emily Luschen - emily.w.luschen-1@ou.edu
# James Ruppert - jruppert@ou.edu
# 5/19/23
# Rosi RB - modified to ingest water paths and to return dask arrays

def precip_class(q_int):
    shape = q_int.shape
    ndims=len(shape)
    shape_out = shape[1:ndims]

    # Integrated water variables
    # Ensure these are Dask arrays if q_int is a Dask array
    LWP = q_int[0]
    IWP = q_int[1]
    # For Q_INT input, q_int[2] is QICE, q_int[3] is QSNOW, q_int[4] is QGRAUP
    # Your original code used q_int[2] for rain, q_int[3] for graupel.
    # Make sure this indexing is consistent with your actual 'q_int' structure.
    # Based on the function docstring: ['QCLOUD', 'QRAIN', 'QICE', 'QSNOW', 'QGRAUP']
    # And your q_int = np.array([ds.lwp, ds.iwp, ds.rwp, ds.gwp]), this means:
    # q_int[0] = lwp (QCLOUD + QRAIN combined in your data?)
    # q_int[1] = iwp (QICE + QSNOW + QGRAUP combined in your data?)
    # q_int[2] = rwp (QRAIN from your data)
    # q_int[3] = gwp (QGRAUP from your data)
    # So, the original function's `q_int[2]` (for rain_thresh) is `rwp` (your q_int[2])
    # and `q_int[3]` (for graup_thresh) is `gwp` (your q_int[3]).
    # This seems consistent, just making sure the comments are aligned.

    RWP = q_int[2] # Rain Water Path
    GWP = q_int[3] # Graupel Water Path

    TWP = LWP + IWP

    # Use xarray.where for masking with Dask arrays
    # Ensure LWP is not zero before division to avoid inf/nan
    cr = xr.where(LWP != 0, IWP / LWP, np.inf) # Use np.inf for where LWP is zero, so cr_thresh condition handles it

    # Threshold parameters (unchanged)
    twp_thresh = 1e-1
    cr_thresh = 2
    graup_thresh = 1e-4
    rain_thresh_conv = 1e-1
    rain_thresh_strat = 1e-2

    # Initialize output array as a Dask array of zeros
    # Use dask.array.zeros or xarray.zeros_like to create a Dask-backed array
    # The shape should be (Time, nCells) after the initial q_int[0] selection
    # Assuming q_int has dimensions (variable, Time, nCells)
    c_type_shape = LWP.shape # Should be (Time, nCells)
    c_type = da.zeros(c_type_shape, dtype=np.int8)

    # Use dask.array.where for efficient boolean indexing with Dask arrays
    # Deep convection
    condition_dc = ((LWP != 0) & (TWP > twp_thresh) & \
                    (cr <= cr_thresh) & \
                    (RWP >= rain_thresh_conv) & \
                    (GWP >= graup_thresh))
    c_type = da.where(condition_dc, 1, c_type)

    # Congestus
    condition_cg = ((LWP != 0) & (TWP > twp_thresh) & \
                    (cr <= cr_thresh) & \
                    (RWP >= rain_thresh_conv) & \
                    (GWP < graup_thresh))
    c_type = da.where(condition_cg, 2, c_type)

    # Shallow
    condition_sc = ((LWP != 0) & (TWP > twp_thresh) & \
                    (cr <= cr_thresh) & \
                    (RWP < rain_thresh_conv))
    c_type = da.where(condition_sc, 3, c_type)

    # Stratiform
    condition_st = ((LWP != 0) & (TWP > twp_thresh) & \
                    (cr > cr_thresh) & \
                    (RWP >= rain_thresh_strat))
    c_type = da.where(condition_st, 4, c_type)

    # Anvil
    condition_an = ((LWP != 0) & (TWP > twp_thresh) & \
                    (cr > cr_thresh) & \
                    (RWP < rain_thresh_strat))
    c_type = da.where(condition_an, 5, c_type)

    return c_type # This will return a Dask array

#### Function to get PE, VMF

In [8]:
### Function to calculate mass-flux based precipitation efficiency ###
def calc_massFlux_precipitationEfficiency(M_u, M_d, areaCell, c_type_dask):

    # ntime = M_u.shape[0]

    # mu_clouds = da.zeros((ntime, 6), dtype=M_u.dtype) # Assuming M_u is a Dask array
    # md_clouds = da.zeros((ntime, 6), dtype=M_u.dtype)
    # count_clouds = da.zeros((ntime, 6), dtype=M_u.dtype) # How many cells contribute to each cloud type
    mu_clouds = da.zeros(6, dtype=M_u.dtype) # Assuming M_u is a Dask array
    md_clouds = da.zeros(6, dtype=M_u.dtype)
    count_clouds = da.zeros(6, dtype=M_u.dtype) # How many cells contribute to each cloud type

    # for it in range(ntime):

    for i in np.arange(1, 7): # Assuming c_type_dask ranges from 1 to 5
    # for i in np.arange(6, 7): # Assuming c_type_dask ranges from 1 to 5

        # Ensure mask is computed as a dask array if c_type_dask is dask
        if i == 6:
            mask = ((c_type_dask == 1) | (c_type_dask == 4) | (c_type_dask == 5)) # Combine DC, ST, AN
        else:
            mask = (c_type_dask == i)

        count = mask.sum()

        # epsilon_mask = epsilon_v2.where(mask)
        Md_mask = M_d.where(mask)
        Mu_mask = M_u.where(mask)
        # areaCell_mask = da.repeat(areaCell[da.newaxis,...], M_d.shape[0]).where(mask) # areaCell should ideally be broadcastable or have matching dims
        areaCell_mask = areaCell.where(mask) # areaCell should ideally be broadcastable or have matching dims

        # It's better to compute the weighted sum and total area, then divide.
        # This will create a dask graph. The .compute() will happen outside the loop
        # if the list is returned, or if you explicitly call .compute() here.
        # For a list, you'd typically gather them later.
        # weighted_sum = (epsilon_mask * areaCell_mask).sum(dim='nCells', skipna=True)
        total_area = areaCell_mask.sum(dim='nCells', skipna=True)
        Mu_weighted_sum = (Mu_mask * areaCell_mask).sum(dim='nCells', skipna=True)
        Md_weighted_sum = (Md_mask * areaCell_mask).sum(dim='nCells', skipna=True)

        # Handle division by zero for total_area to avoid NaN/inf
        # epsilon_mask_mean_v2 = (weighted_sum / total_area).where(total_area != 0)
        Mu_mask_mean = (Mu_weighted_sum / total_area).where(total_area != 0)
        Md_mask_mean = (Md_weighted_sum / total_area).where(total_area != 0)

        # Finally, calculate epsilon for cloud type with averaged Md, Mu
        # V1
        # epsilon_mask_mean_v1 = 1.0 - (Md_mask_mean / Mu_mask_mean).where(Mu_mask_mean != 0) # Avoid inf/NaN where M_d is zero
        # epsilon_clouds.append(epsilon_mask_mean_v1) # Append dask.array.Array objects
        # count_clouds.append(count)
        # mu_clouds.append(Mu_mask_mean)
        # md_clouds.append(Md_mask_mean)
        count_clouds[i-1] = count
        mu_clouds[i-1] = Mu_mask_mean
        md_clouds[i-1] = Md_mask_mean

        # Put V2 after V1
        # epsilon_clouds.append(epsilon_mask_mean_v2) # Append dask.array.Array objects

    # finally, domain-mean
    # If you want to compute the domain mean of epsilon for the whole domain (not by cloud type)
    # domain_mean_epsilon = (epsilon * areaCell).sum(dim='nCells', skipna=True) / areaCell.sum(dim='nCells', skipna=True)
    # You could return both epsilon_clouds and domain_mean_epsilon, or whatever makes sense for your output.
    return [count_clouds, mu_clouds, md_clouds] # This will be a list of Dask DataArray 

### Main driver loop

#### Get file list

In [9]:
# Get list of desired file times
file_times_arr = np.arange('2000-05-01T06:00:00', '2000-05-11T06:00:00', 6, dtype='datetime64[h]')
file_times = [file_times_arr[i].astype('datetime64[D]').astype(str)+'_'+str(file_times_arr[i]).split('T')[1].split(':')[0]+'.00.00' for i in range(len(file_times_arr))]

istart_set=11
file_times[istart_set]

'2000-05-04_00.00.00'

#### Start loops

In [10]:
%%time
# Main loop
import pickle

pclass_names = ['DC', 'CG', 'SC', 'ST', 'AN', 'DSA']

print('Starting loop...')
nCells_chunk_size = 100000

exp_names = ["CTL", "HOMO_RAD", "CLIM_RAD"]

main_path = "/glade/campaign/mmm/dpm/rberrios/glade_scratch/MPAS_APE/aqua_sstmax10N_ASD/"

pickle_dir = '/glade/derecho/scratch/ruppert/tc-crfrad/pickle_out/'

# for expName in exp_names:
for expName in exp_names[2:]:

    data_path = f"{main_path}{expName}/TC_3km/"
    scdir = '/glade/derecho/scratch/ruppert/tc-crfrad/mpas/'+expName+'/'

    # data_path = f"/glade/campaign/mmm/dpm/rberrios/glade_scratch/MPAS_APE/aqua_sstmax10N_ASD/{expName}/TC_3km/"

    # Open the dataset with dask backend. This loads lazily.
    # Specify chunks to optimize memory usage and parallel processing.
    # You'll need to know typical chunk sizes for your variables, or let xarray guess.
    # For large datasets, manual chunking can be critical.
    # Example: If 'Time' dimension is large, chunk it. 'nCells' might be good to chunk too.
    # ds = xr.open_mfdataset(data_path + "waterPaths*", combine="nested", concat_dim="Time",
    #                        chunks={'Time': 'auto', 'nCells': 'auto'}) # 'auto' lets Dask guess
    # Or specify explicitly, e.g., {'Time': 24, 'nCells': 1000}

    if expName == "CTL":
        istart = 0
    elif expName == "HOMO_RAD":
        istart = 0
    elif expName == "CLIM_RAD":
        istart = istart_set

    # for time in file_times:
    # for time in file_times[istart:]:
    for time in file_times[istart:]:

        # print(f"Processing {expName} for time {time}")
        print(f"Processing file: VMF_pclass_{expName}_{time}_{str(latbounds[0])}-{str(latbounds[1])}.pickle")

        # print('opening files')
        # wp_files = [data_path+'waterPaths.'+time+'.nc' for time in file_times]
        wp_files = data_path+'waterPaths.'+time+'.nc'
        # ds = xr.open_mfdataset(wp_files,
        ds = xr.open_mfdataset(wp_files,
                    # combine="nested", concat_dim="Time", 
                    parallel=True, 
                    chunks={"Time": -1, "nCells": nCells_chunk_size})

        # vmf_files = [scdir+'vmfs.'+time+'.nc' for time in file_times]
        vmf_files = scdir+'vmfs.'+time+'.nc'
        # ds_vmf = xr.open_mfdataset(vmf_files,
        ds_vmf = xr.open_mfdataset(vmf_files,
                    # combine="nested", concat_dim="Time", 
                    parallel=True,
                    chunks={"Time": -1, "nCells": nCells_chunk_size})

        # print()
        # print('subsetting')
        # Select cells within latitude range. This operation is also lazy if `ds` is Dask-backed.
        ds_tropical = ds.isel(nCells=ind_within_lat)
        ds_vmf_tropical = ds_vmf.isel(nCells=ind_within_lat)

        # print('reading variables')
        # Convert to a list of DataArrays, and wrap in dask.array.stack to create a single Dask array
        # This creates a Dask-backed array 'q_int_dask' without loading data into memory yet.
        q_int_dask = da.stack([
            ds_tropical.lwp.data,
            ds_tropical.iwp.data,
            ds_tropical.rwp.data,
            ds_tropical.gwp.data
        ], axis=0) # Stack along a new 0th dimension for the different water paths

        mu = ds_vmf_tropical.mu
        md = ds_vmf_tropical.md

        # print('classifying')
        # Call the classification function. This will return a Dask array (c_type_dask).
        # The actual computation of c_type is still lazy at this point.
        c_type_dask = precip_class(q_int_dask)

        # print('getting PE')
        # Get Mu, Md as a function of PClass
        vmf_pclass = calc_massFlux_precipitationEfficiency(mu, md, areaCell_tropical, c_type_dask[0])

        # print('daks.compute for VMFs')
        results = dask.compute(vmf_pclass)[0] # dask.compute returns a tuple of results
        count_results = results[0]
        mu_results = results[1]
        md_results = results[2]

        # Write out to pickle

        # pickle_file_out = f"{scdir}PE_massFlux_{expName}_{time}.pickle"
        pickle_file_out = f"{pickle_dir}VMF_pclass_{expName}_{time}_{str(latbounds[0])}-{str(latbounds[1])}.pickle"
        # pickle_file_out = f"{pickle_dir}VMF_{expName}.pickle"
        with open(pickle_file_out, 'wb') as f:
            # pickle.dump(PE_thisExp, f)
            pickle.dump([count_results, mu_results, md_results], f)

    print(f"Finished processing {expName}")

print('Classification complete.')

Starting loop...
Processing file: VMF_pclass_CLIM_RAD_2000-05-04_00.00.00_15-20.0.pickle


/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 23.42 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Processing file: VMF_pclass_CLIM_RAD_2000-05-04_06.00.00_15-20.0.pickle


/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 23.42 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Processing file: VMF_pclass_CLIM_RAD_2000-05-04_12.00.00_15-20.0.pickle


...

Processing file: VMF_pclass_CLIM_RAD_2000-05-10_18.00.00_15-20.0.pickle


/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 23.42 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Processing file: VMF_pclass_CLIM_RAD_2000-05-11_00.00.00_15-20.0.pickle


/glade/work/ruppert/conda-envs/plotting/lib/python3.11/site-packages/distributed/client.py:3371: UserWarning: Sending large graph of size 23.42 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


Finished processing CLIM_RAD
Classification complete.
CPU times: user 1h 54min 58s, sys: 18min 51s, total: 2h 13min 49s
Wall time: 2h 9min 13s


### Close cluster

In [ ]:
cluster.close()